In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
import pathlib

data_dir = pathlib.Path('/kaggle/input/nail-disease-image-classification-dataset/nail_disease_dataset')

In [ ]:
print(data_dir)

In [ ]:
healthy_images = list(data_dir.glob('train/healthy/*'))
print(healthy_images[0])
PIL.Image.open(str(healthy_images[0]))

In [ ]:
from PIL import Image
# For example, we can take images from the training set for the 'healthy' category
healthy_images = list(data_dir.glob('train/healthy/*'))

# Open the first image from the 'healthy' category
img = Image.open(str(healthy_images[1]))

# Check the image size
width, height = img.size
print(f"Image size: {width}x{height} pixels")

In [ ]:
size = 224
img_height,img_width=size,size
batch_size=32
path = pathlib.Path("/kaggle/input/nail-disease-image-classification-dataset/nail_disease_dataset/train")
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  path,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
import pathlib
import tensorflow as tf

# Parameters
img_height, img_width = size, size
batch_size = 32

# Defining the path to the data directory
data_dir = pathlib.Path('/kaggle/input/nail-disease-image-classification-dataset/nail_disease_dataset')

# Loading training data divided into validation
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir / 'train',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir / 'train',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# Display class names
class_names = train_ds.class_names
print(class_names)

# Loading test data
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir / 'test',
    image_size=(img_height, img_width),
    batch_size=batch_size)

# Optional data loading acceleration
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Initialize the model
pretrained_model = tf.keras.applications.MobileNetV3Large(
    include_top=False,
    input_shape=(size, size, 3),
    pooling='avg',
    weights='/kaggle/input/mobileenettv3/weights_mobilenet_v3_large_224_1.0_float_no_top_v2.h5'
)

# Set the layers of the pretrained model to untrained
for layer in pretrained_model.layers:
    layer.trainable = False

# Create a sequential model
model = Sequential()
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Build model using sparse_categorical_crossentropy
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training
epochs = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_ds, verbose=2)
print(f'Test accuracy: {test_accuracy}')

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
# Save the trained model to a specified path
model.save('/kaggle/working/nail_disease_classification_model.h5')

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix

# Load class names
class_names = ['healthy', 'onychomycosis', 'psoriasis']

# Image parameters
img_height, img_width = size, size

# Function for loading and processing the image
def load_and_preprocess_image(image_path):
    image = cv2.imread(str(image_path))
    image_resized = cv2.resize(image, (img_height, img_width))
    image_array = np.expand_dims(image_resized, axis=0)
    return image_array

# Path to the directory with test data
data_dir = Path('/kaggle/input/nail-disease-image-classification-dataset/nail_disease_dataset/test')

# Preparing lists to store real labels and predicted labels
y_true = []
y_pred = []

for class_name in class_names:
    class_dir = data_dir / class_name
    image_paths = list(class_dir.glob('*'))

    for image_path in image_paths:
        # Load and process the image
        image = load_and_preprocess_image(image_path)

        # Class prediction
        pred = model.predict(image)

        # Get the index of the predicted class
        predicted_class = np.argmax(pred)

        # Adding actual label and predicted label to lists
        y_true.append(class_names.index(class_name))
        y_pred.append(predicted_class)

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Display results
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))